In [1]:
import pandas as pd

#Importo le informazioni riguardanti le qualità delle tasks 
tasks = pd.read_csv("../Dataset 2.0/Tasks.csv")
tasks = tasks.drop(0, axis = 0)
tasks = tasks.drop("Deadline", axis = 1)


#Importo i coefficienti di regressione di ogni agente
coeffs = pd.read_csv("../Coefficienti di Regressione/coeffs.csv")

decisions = pd.read_csv("../Dataset 2.0/Decisions.csv")

#Selezionare il Session ID ed il Round voluti
#Assicurarsi di selezionare lo stesso round in "RegressionModel.py" e in "OptimizationModel.py"
decisions = decisions[decisions["Session ID"] == "e7e52776-2750-4cde-a5da-093ccb8feaf9"]
decisions = decisions[decisions["Round"] == 1]
decisions = decisions.sort_values("Worker Agent ID")
coeffs

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,1.135297e-08,2.895223e-09,-5.525225e-09,7.610982e-09,-1.040070e-09,-4.327087e-09,4.564795e-09,-2.731863e-09,-1.225825e-09,-1.333201e-09,7.771331e-10,1.029156e-08,-6.599144e-09,-5.991374e-10,8.095032e-10,-3.547793e-09,2.438035e-09,8.025340e-10,-5.568801e-10,-1.865818e-09
1,-1.467271e-03,-6.051352e-03,2.910450e-04,1.088817e-03,2.351017e-03,2.802431e-03,3.383489e-03,4.996739e-03,5.676590e-03,2.933651e-03,-6.638590e-03,-6.619619e-03,-2.803775e-03,4.317027e-04,-2.020762e-03,9.169540e-04,3.384114e-03,4.435585e-03,5.338799e-03,1.232925e-03
2,-2.934543e-04,-1.210270e-03,5.820901e-05,2.177633e-04,4.702033e-04,5.604861e-04,6.766979e-04,9.993478e-04,1.135318e-03,5.867302e-04,-1.327718e-03,-1.323924e-03,-5.607550e-04,8.634053e-05,-4.041525e-04,1.833908e-04,6.768228e-04,8.871171e-04,1.067760e-03,2.465849e-04
3,-1.467271e-03,-6.051352e-03,2.910450e-04,1.088817e-03,2.351017e-03,2.802431e-03,3.383489e-03,4.996739e-03,5.676590e-03,2.933651e-03,-6.638590e-03,-6.619619e-03,-2.803775e-03,4.317027e-04,-2.020762e-03,9.169540e-04,3.384114e-03,4.435585e-03,5.338799e-03,1.232925e-03
4,-6.104951e-03,2.825299e-03,1.157873e-02,1.648174e-02,1.207603e-02,1.153152e-02,1.066671e-02,1.071478e-02,9.064550e-03,7.261920e-03,-2.357339e-03,3.108167e-03,5.176090e-03,1.089036e-02,1.415033e-02,1.227756e-02,9.092171e-03,1.061560e-02,9.175133e-03,3.269367e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,-5.604477e-03,8.582621e-04,1.400089e-02,1.681026e-02,1.694526e-02,1.474596e-02,1.113068e-02,9.048374e-03,9.588731e-03,9.501525e-03,-7.889762e-03,-1.462658e-03,1.068692e-02,1.543930e-02,1.389201e-02,1.374094e-02,1.272761e-02,1.071478e-02,6.892532e-03,4.133913e-03
89,-1.120895e-03,1.716524e-04,2.800178e-03,3.362052e-03,3.389051e-03,2.949192e-03,2.226137e-03,1.809675e-03,1.917746e-03,1.900305e-03,-1.577952e-03,-2.925317e-04,2.137383e-03,3.087859e-03,2.778401e-03,2.748189e-03,2.545522e-03,2.142956e-03,1.378506e-03,8.267827e-04
90,-5.604477e-03,8.582621e-04,1.400089e-02,1.681026e-02,1.694526e-02,1.474596e-02,1.113068e-02,9.048374e-03,9.588731e-03,9.501525e-03,-7.889762e-03,-1.462658e-03,1.068692e-02,1.543930e-02,1.389201e-02,1.374094e-02,1.272761e-02,1.071478e-02,6.892532e-03,4.133913e-03
91,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [22]:
from mip import *

m = Model()

#DATI

#Identifier del Round
Round = list(decisions["ID"])


#Peso dell'Identifier del round
wRound = list(coeffs.iloc[0])


#Value di ogni task
value = []
for i in tasks["Value"]:
    value.append(int(i))


#Peso del Value (coeffs 1, 4, 7, ..., 88)



#Complessità di ogni task
difficulty = []
for i in tasks["Difficulty"]:
    difficulty.append(float(i))


#Peso della Complessità (coeffs 2, 5, 8, ..., 89)



#Stamina consumata da ogni task
effort = []
for i in tasks["Effort Required"]:
    effort.append(int(i))

    

#Peso della Stamina consumata (coeffs 3, 6, 9, ..., 90)



#Identifier del Worker Agent
WAID = list(decisions["Worker Agent ID"])


#Peso dell'Identifier del WA
wWAID = list(coeffs.iloc[91])


#Reputazione del WA prima del round
Rep = list(decisions["Worker Agent Reputation"])


#Peso della reputation
wRep = list(coeffs.iloc[92])

#creo i range utili per la creazione di variabili
#(A, T) = (range(len(WAID)), range(len(value)))


#VARIABILI

#x[a, t] vale 1 se la task t è assegnata al WA a, 0 altrimenti
#x = [[m.add_var(var_type=BINARY) for t in T] for a in A]

#y(a) contiene la reputation del WA a fine round
#y = [m.add_var(var_type=CONTINUOUS) for a in A]


#VINCOLI

#Vincolo che impone che ogni task venga assegnata ad uno ed un solo WA
#for t in T:
#    m += xsum(x[a][t] for a in A) == 1


#OBIETTIVO

#Massimizzare la sommatoria delle reputation dei worker agent


#m.write("model.lp")
#m.optimize()